Тема “Генерация текстов (языковое моделирование)”

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов


In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path
from keras.callbacks import EarlyStopping

In [2]:
%%capture
!wget https://www.dropbox.com/s/wfrfgaizixxwz3w/evgenyi_onegin.txt

In [3]:
data_path = Path('evgenyi_onegin.txt')
text = data_path.read_text()
print(f'text lenght is {len(text)} chars')
print(f'text sample:\n {text[:500]}')

text lenght is 286984 chars
text sample:
 Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [4]:
text = text + text

In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters: \n{np.array(vocab+[""]).reshape(4, -1)}')

131 unique characters: 
[['\n' ' ' '!' '"' "'" '(' ')' ',' '-' '.' '0' '1' '2' '3' '4' '5' '6'
  '7' '8' '9' ':' ';' '?' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'L']
 ['M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'V' 'W' 'X' 'Y' 'a' 'b' 'c' 'd' 'e' 'f'
  'g' 'h' 'i' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v']
 ['w' 'y' 'z' '{' '}' 'А' 'Б' 'В' 'Г' 'Д' 'Е' 'Ж' 'З' 'И' 'К' 'Л' 'М' 'Н'
  'О' 'П' 'Р' 'С' 'Т' 'У' 'Ф' 'Х' 'Ц' 'Ч' 'Ш' 'Ь' 'Э' 'Ю' 'Я']
 ['а' 'б' 'в' 'г' 'д' 'е' 'ж' 'з' 'и' 'й' 'к' 'л' 'м' 'н' 'о' 'п' 'р' 'с'
  'т' 'у' 'ф' 'х' 'ц' 'ч' 'ш' 'щ' 'ъ' 'ы' 'ь' 'э' 'ю' 'я' '']]


In [6]:
# мэппинги 'символ <-> индекс'
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [7]:
text_as_int = np.array([char2idx[c] for c in text])
list(zip(text_as_int, text))[:20]

[(71, 'А'),
 (110, 'л'),
 (104, 'е'),
 (109, 'к'),
 (116, 'с'),
 (99, 'а'),
 (112, 'н'),
 (103, 'д'),
 (115, 'р'),
 (1, ' '),
 (87, 'С'),
 (104, 'е'),
 (115, 'р'),
 (102, 'г'),
 (104, 'е'),
 (104, 'е'),
 (101, 'в'),
 (107, 'и'),
 (122, 'ч'),
 (1, ' ')]

In [8]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
# проверка обратного преобразования тензора
[idx2char[i.numpy()] for i in char_dataset.take(10)]

['А', 'л', 'е', 'к', 'с', 'а', 'н', 'д', 'р', ' ']

In [9]:
# The maximum length sentence for a single input in characters
seq_length = 100

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
[idx2char[i.numpy()] for i in sequences.take(1)]

[array(['А', 'л', 'е', 'к', 'с', 'а', 'н', 'д', 'р', ' ', 'С', 'е', 'р',
        'г', 'е', 'е', 'в', 'и', 'ч', ' ', 'П', 'у', 'ш', 'к', 'и', 'н',
        '\n', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'Е', 'в', 'г', 'е', 'н',
        'и', 'й', ' ', 'О', 'н', 'е', 'г', 'и', 'н', '\n', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], dtype='<U1')]

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [12]:
# Batch size
BATCH_SIZE = 64
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000
EPOCHS = 20

In [13]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

# loss function sparse_categorical_crossentropy
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

Сетка с GRU

In [15]:
class GRUgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(GRUgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)

        x = (x + x1)/2
        return self.fc(x)

model_gru = GRUgenerator(vocab_size, embedding_dim, BATCH_SIZE)

model_gru.compile(optimizer='adam', loss=loss)

In [16]:
history = model_gru.fit(dataset, epochs=EPOCHS)

Epoch 1/20
88/88 [==============================] - 40s 243ms/step - loss: 2.0685
Epoch 2/20
88/88 [==============================] - 22s 247ms/step - loss: 1.5170
Epoch 3/20
88/88 [==============================] - 23s 250ms/step - loss: 1.3515
Epoch 4/20
88/88 [==============================] - 23s 252ms/step - loss: 1.2828
Epoch 5/20
88/88 [==============================] - 23s 255ms/step - loss: 1.2390
Epoch 6/20
88/88 [==============================] - 23s 258ms/step - loss: 1.1940
Epoch 7/20
88/88 [==============================] - 23s 258ms/step - loss: 1.1578
Epoch 8/20
88/88 [==============================] - 23s 255ms/step - loss: 1.1172
Epoch 9/20
88/88 [==============================] - 23s 257ms/step - loss: 1.0754
Epoch 10/20
88/88 [==============================] - 23s 255ms/step - loss: 1.0663
Epoch 11/20
88/88 [==============================] - 23s 256ms/step - loss: 1.0254
Epoch 12/20
88/88 [==============================] - 23s 255ms/step - loss: 0.9752
Epoch 13/20
8

In [17]:
def generate_text(model, start_string, temperature=0.5):
    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.

    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [18]:
text_ = generate_text(model_gru, start_string=u"И вот идет уже ",  temperature=0.7)
print(text_)

И вот идет уже вещебныелевовенедыхосле                     повобрашетой                 .
                                                                стиской  жнегодомо                                                     омиморанеей                                                         дит                      инострамей                                                 селитья                                   ст                                                              т           ней                 


Сетка с LSTM

In [19]:
def build_lstm_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [20]:
model_lstm = build_lstm_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [21]:
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 128)           16768     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          4722688   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense_1 (Dense)             (64, None, 131)           134275    
                                                                 
Total params: 30,051,843
Trainable params: 30,051,843
No

In [22]:
model_lstm.compile(optimizer='adam', loss=loss)

In [23]:
!rm -rf ./training_checkpoints

In [24]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = Path('./training_checkpoints')
# Name of the checkpoint files
checkpoint_prefix = checkpoint_dir / "ckpt_{epoch}"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model_lstm(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [27]:
history = model_lstm.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
88/88 [==============================] - 33s 328ms/step - loss: 2.4045
Epoch 2/20
88/88 [==============================] - 30s 331ms/step - loss: 2.0473
Epoch 3/20
88/88 [==============================] - 31s 347ms/step - loss: 1.6986
Epoch 4/20
88/88 [==============================] - 30s 336ms/step - loss: 1.5426
Epoch 5/20
88/88 [==============================] - 30s 340ms/step - loss: 1.4445
Epoch 6/20
88/88 [==============================] - 32s 351ms/step - loss: 1.3884
Epoch 7/20
88/88 [==============================] - 31s 339ms/step - loss: 1.3614
Epoch 8/20
88/88 [==============================] - 30s 339ms/step - loss: 1.3356
Epoch 9/20
88/88 [==============================] - 31s 349ms/step - loss: 1.3464
Epoch 10/20
88/88 [==============================] - 30s 337ms/step - loss: 1.3178
Epoch 11/20
88/88 [==============================] - 30s 337ms/step - loss: 1.2937
Epoch 12/20
88/88 [==============================] - 31s 350ms/step - loss: 1.2787
Epoch 13/20
8

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)
model_lstm = build_lstm_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model_lstm.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model_lstm.build(tf.TensorShape([1, None]))

In [29]:
model_lstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (1, None, 128)            16768     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           4722688   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_2 (Dense)             (1, None, 131)            134275    
                                                                 
Total params: 30,051,843
Trainable params: 30,051,843


In [30]:
text_ = generate_text(model_lstm, start_string=u"И вот идет уже ",  temperature=0.7)
print(text_)

И вот идет уже тросья
                                                                                                        XXII

                        Быгда вослылиго                 Сморез от содую плечные,
                        Лабор ужели болибольной,
                        Вестыхча возледний опревелкий
                        В опливо с тлег и наде всем
                        Дегот марочный невовран;
                                                                                    И пладе плеври
